In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets, models
from torchvision.transforms import ToTensor, Pad, Compose,CenterCrop, ToPILImage, Normalize, ConvertImageDtype, Resize
from torch import nn
from torch.nn import init, Linear, ReLU, Softmax
from torch.nn.init import xavier_uniform_
from torch.optim import SGD, Adam, lr_scheduler
import torch.nn.functional as F
import pandas as pd 
import os
import time
import copy
import matplotlib.pyplot as plt
from torchvision.models import resnet18
from tqdm import tqdm
from functools import partial
tqdm = partial(tqdm, position=0, leave=True)

In [ ]:
#Dataset

from torchvision.io import read_image

class GTSRB(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)[["Path","ClassId"]]
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        label = self.img_labels.iloc[idx, 1]
        image = read_image(img_path)
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [ ]:
#Dataset Initialization
img_dir = "/kaggle/input/gtsrb-german-traffic-sign/"
train_file = "/kaggle/input/gtsrb-german-traffic-sign/Train.csv"
full_train_data = GTSRB(img_dir = img_dir, annotations_file = train_file,
                   transform = Compose([Resize((30,30)), ConvertImageDtype(torch.float32)]))
train_data, val_data = torch.utils.data.random_split(full_train_data, [len(full_train_data)-256, 256])

#Prepare DataLoader
from torch.utils.data import DataLoader
b_size = 256
train_dataloader = DataLoader(train_data, batch_size=b_size, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=b_size, shuffle=False)

In [ ]:
# Display image and label.
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0]
label = train_labels[0]
img = ToPILImage()(img).convert("RGB")
plt.imshow(img)
plt.show()
print(f"Label: {label}")

In [ ]:
#Model artchitecture, define new model here
class conv_net(nn.Module):
    def __init__(self):
        super(conv_net,self).__init__()
        
    def forward(self, X):
        
        
        return X

In [ ]:
#Training Module
def train_model(model, train_dataloader, val_dataloader, criterion, optimizer, scheduler, num_epochs=30):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
                dataloader = train_dataloader
            else:
                model.eval()   # Set model to evaluate mode
                dataloader = val_dataloader
            
            dataset_size = len(dataloader)*b_size
            pbar = tqdm(total=dataset_size)

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)    
                    _, preds = torch.max(outputs.data, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                pbar.update(len(labels))
    
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels)
                    
            epoch_loss = running_loss / dataset_size
            epoch_acc = running_corrects.double() / dataset_size
            
            if phase == 'train':
                scheduler.step()

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = running_corrects.double() / dataset_size
                best_model_wts = copy.deepcopy(model.state_dict())
                

        print()
        
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
#Training Setting 

##Model init
#model = conv_net() ##using new model
model = resnet18(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 43)

criterion = nn.CrossEntropyLoss(reduction = "mean")
optim = Adam(model.parameters(), lr = 0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optim, step_size=5, gamma=0.1)

#put on cuda if possible
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
nb_epochs = 10

In [ ]:
#Train model
torch.backends.cudnn.benchmark = True
model = train_model(model,train_dataloader, val_dataloader, criterion, optim, exp_lr_scheduler,
                       num_epochs=nb_epochs)

In [ ]:
torch.save(model, "/kaggle/working/ResNet18_best.pth")